# <a id='toc1_'></a>[PARKINSON DETECTION BY SNIFFER DOGS](#toc0_)



**Table of contents**<a id='toc0_'></a>    
- [PARKINSON DETECTION BY SNIFFER DOGS](#toc1_)    
- [LIBRARIES](#toc2_)    
- [REGISTER DATA SET](#toc3_)    
- [CLASS NAMES](#toc4_)    
- [SETUP THE MODEL FOR INFERENCE](#toc5_)    
- [INFERENCE ON IMAGES](#toc6_)    
- [INFERENCE ON VIDEO](#toc7_)    
- [START AND STOP SNIFFING](#toc8_)    
- [SAVE SNIFFING SNIPPETS](#toc9_)    
- [CSV](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc2_'></a>[LIBRARIES](#toc0_)

In [97]:
!python --version
print("")

# Torch
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
print(f"Torch version: {TORCH_VERSION}")


# Cuda
CUDA_VERSION = torch.__version__.split("+")[-1]
print(f"cuda version: {CUDA_VERSION}")
print("Torch cuda is available :", torch.cuda.is_available())

print("")
!nvcc --version

# Detectron2
import detectron2
print("")
print("detectron2:", detectron2.__version__)

from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader, DatasetMapper
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultPredictor

from detectron2.utils.visualizer import Visualizer, ColorMode

# Cv2 
# conda install -c conda-forge opencv
import cv2
print("cv2:", cv2.__version__)

# Scipy
# pip install scipy
import scipy
print("Scipy version:", scipy.__version__)

# Sklearn
# pip install scikit-learn
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# Import CountVectorizer to create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

# To Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

#import to use an TF-IDF Vectorizer Object, to Remove all english stop words such as 'the', 'a'
from sklearn.feature_extraction.text import TfidfVectorizer
from ast import literal_eval

# and other libraries
import os
import glob

import pandas as pd
import numpy as np

from PIL import Image

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

import random
import json
import pickle
import requests
import time 
import shutil
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET

import logging # I use this to have more information about what's going on
logging.basicConfig()
log = logging.getLogger()
log.setLevel('INFO')


import cv2
import os
import math
import moviepy as mp

Python 3.13.2

Torch version: 2.6
cuda version: cu126
Torch cuda is available : True

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:45:30_PST_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0

detectron2: 0.6
cv2: 4.11.0
Scipy version: 1.15.2


# TO DO 
update the video numbers in the list below and press run all 

In [ ]:
list_to_do = [2630,2631,2632,2634,2635,2636,2637,2638,2639, 2640,2641,2642,2643,2644,2645]



# <a id='toc3_'></a>[REGISTER DATA SET](#toc0_)

In [98]:
coco = "HeadAndTankDetect.v4i.coco/train/_annotations.coco.json"
images = "HeadAndTankDetect.v4i.coco/train/"

source_dir = images

#Register datasets
register_coco_instances('train', {}, os.path.join(source_dir, 'train.json'), source_dir)
register_coco_instances('val', {}, os.path.join(source_dir, 'val.json'), source_dir)

#Get the internal representation
train_dicts = DatasetCatalog.get("train")
test_dicts = DatasetCatalog.get("val")

#Get the metadata
train_metadata = MetadataCatalog.get("train")
test_metadata = MetadataCatalog.get("val")

print(f"Registered objects")

print("Number of images in train set: ")
print(len(DatasetCatalog.get("train")))
print("Number of images in val set: ")
print(len(DatasetCatalog.get("val")))

WARNING [05/27 21:42:23 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/27 21:42:23 d2.data.datasets.coco]: Loaded 1807 images in COCO format from HeadAndTankDetect.v4i.coco/train/train.json
WARNING [05/27 21:42:23 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/27 21:42:23 d2.data.datasets.coco]: Loaded 719 images in COCO format from HeadAndTankDetect.v4i.coco/train/val.json
Registered objects
Number of images in train set: 
WARNING [05/27 21:42:23 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/27 21:42:23 d2.data.datasets.coco]: Loaded 1807 images in COCO format from HeadAndTankDetect.v4i.coco/train/train.json
1807
Number of images in val set: 
WARNING [05/27 21:42:24 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for yo

# <a id='toc4_'></a>[CLASS NAMES](#toc0_)

In [99]:
print(test_metadata)
print("Class names:", train_metadata.get("thing_classes"))
nr_classes = len(train_metadata.get("thing_classes"))

Metadata(name='val', json_file='HeadAndTankDetect.v4i.coco/train/val.json', image_root='HeadAndTankDetect.v4i.coco/train/', evaluator_type='coco', thing_classes=['Tank', 'Head', 'Tank', 'assistant', 'nose'], thing_dataset_id_to_contiguous_id={0: 0, 1: 1, 2: 2, 3: 3, 4: 4})
Class names: ['Tank', 'Head', 'Tank', 'assistant', 'nose']


# <a id='toc5_'></a>[SETUP THE MODEL FOR INFERENCE](#toc0_)

In [100]:

def setup_detectron2(nr_classes=nr_classes):

    model =  "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(model))

    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    #cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  

    cfg.MODEL.ROI_HEADS.NUM_CLASSES = nr_classes  
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3
    # set threshold for this model : 
    #Higher value (0.8) = More selective, only very confident predictions remain
    #Lower value (e.g., 0.5) = More permissive, includes less confident predictions
    predictor = DefaultPredictor(cfg)

    return predictor


# <a id='toc7_'></a>[INFERENCE ON VIDEO](#toc0_)

In [101]:
# Process video and save output
def process_video(input_video_path, output_video_path):

    frame_count = 0
    jar_detection_data = []
    bounding_box_location_tank_head = []

    predictor = setup_detectron2()
    cap = cv2.VideoCapture(input_video_path)

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID', 'MJPG', etc.
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform object detection
        outputs = predictor(frame)

        # Draw bounding boxes
        instances = outputs["instances"].to("cpu")

        boxes = instances.pred_boxes.tensor.numpy()               
        scores = instances.scores.numpy()
        classes = instances.pred_classes.numpy()

        # Save the instances for statistics & store timestamps

        if 4 in instances.pred_classes : 
            timestamp = frame_count / fps  
        
            entry = {
                "frame_number": frame_count,
                "timestamp": timestamp,
                "bounding_box": boxes.tolist(),
                "confidence": instances.scores.tolist(),
                "pred_classes": instances.pred_classes.tolist()
            }  

        else: 
            timestamp = frame_count / fps  
        
            entry = {
                "frame_number": frame_count,
                "timestamp": timestamp,
                "bounding_box": boxes.tolist(),
                "confidence": instances.scores.tolist(),
                "pred_classes": instances.pred_classes.tolist()
            }


        bounding_box_location_tank_head.append(entry)
        frame_count += 1



        for box, score, cls in zip(boxes, scores, classes):
            if score > 0.5:  # Confidence threshold
                x1, y1, x2, y2 = map(int, box)

                class_name = train_metadata.thing_classes[cls]
                label = f"{class_name}: {score:.2f}"

                if cls == 1:  # class 1 is 'head'
                    continue
                    # cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 170, 0), 2)
                    # cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 170, 0), 2)

                elif cls == 4:  #  class 4 is 'nose'
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (170, 170, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (170, 170, 0), 2)

                elif cls == 2:  #  class 2 is 'tank'
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (170, 0, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (170, 0, 0), 2)
                
                else:                
                    continue

        # Write the frame
        out.write(frame)


    # Release everything
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    return bounding_box_location_tank_head, fps
    



# VIDEO NAMES + INPUT + OUTPUTFOLDER + DURATION

In [102]:

input_video_path = "Videos_Cut/DSCN2630.mp4"

vid_nr = input_video_path[-8:-4]
print("Video number:", vid_nr)


f = "SniffingSnippetsIsabelle/"
output_video_name = vid_nr + 'processed_output.mp4'
output_video_path = os.path.join(f, output_video_name)

# duration of the video in seconds
input_video_duration = mp.VideoFileClip(input_video_path).duration
print("Input video duration:", input_video_duration, "seconds")



Video number: 2630
Input video duration: 26.09 seconds


The duration of a video in seconds is directly related to the total frame count and the frame rate (frames per second, or fps) by the following formula:

Duration (seconds) = Total Frame Count / FPS
 
Total Frame Count: The number of individual frames in the video.

FPS (Frames Per Second): How many frames are displayed each second.

For example, if a video has 600 frames and a frame rate of 10 fps, the duration is:

600 / 10 = 60  seconds

This relationship is fundamental in video processing and editing. However, it's important to note that in some video formats or editing scenarios, the actual frame count may not exactly equal fps × duration due to rounding, dropped frames, or variable frame rates. For most standard videos with a constant frame rate, the formula above holds true.

# FUNCTION BOUNDING BOXES  

In [103]:
def select_bboxes(bboxes, indices):
    return [bboxes[i] for i in indices if i < len(bboxes)]

# FUNCTION OVERLAP

In [104]:
def bbox_overlap(box1, box2):
    return (box1[0] < box2[2] and box1[2] > box2[0]) and \
           (box1[1] < box2[3] and box1[3] > box2[1])

# FUNCTION START STOP SNIFFING

In [105]:
def start_stop_sniffing(bounding_box_location_tank_head) :
    bb = bounding_box_location_tank_head.copy()
    bb = pd.DataFrame(bb)

    bb['indexes_head'] = bb["pred_classes"].apply(lambda x: [i for i, v in enumerate(x) if v == 1])
    bb['indexes_tank'] = bb["pred_classes"].apply(lambda x: [i for i, v in enumerate(x) if v == 2])
    bb['indexes_nose'] = bb["pred_classes"].apply(lambda x: [i for i, v in enumerate(x) if v == 4])

    bb['bboxes_head'] = bb.apply(lambda row: select_bboxes(row['bounding_box'], row['indexes_head']), axis=1)
    bb['bboxes_tanks'] = bb.apply(lambda row: select_bboxes(row['bounding_box'], row['indexes_tank']), axis=1)
    bb['bboxes_nose'] = bb.apply(lambda row: select_bboxes(row['bounding_box'], row['indexes_nose']), axis=1)

    bb['sniffing'] = bb.apply(
        lambda row: (
            'sniffing' if any(
                bbox_overlap(n, t)
                for n in row['bboxes_nose']
                for t in row['bboxes_tanks']
            ) else 'not_sniffing'
        ),
        axis=1
    )

    bb['prev_sniffing'] = bb['sniffing'].shift(1)
    bb['next_sniffing'] = bb['sniffing'].shift(-1)

    bb['start-stop'] = "continue"
    if bb['prev_sniffing'] is None:
        bb['start-stop'] = 'start'



    # sniffing starts when previous behavior was nothing or not sniffing
    bb['start-stop'] = bb.apply(
        lambda row: 'start' if row['prev_sniffing'] is None and row['sniffing'] == 'sniffing' else row['start-stop'], axis=1
    )

    bb['start-stop'] = bb.apply(
        lambda row: 'start' if row['prev_sniffing'] != 'sniffing' and row['sniffing'] == 'sniffing' else row['start-stop'], 
         axis=1
    )

    # sniffing stops when current is not sniffing and previous was sniffing
    bb['start-stop'] = bb.apply(
        lambda row: 'stop' if row['sniffing'] != 'sniffing' and row['prev_sniffing'] == 'sniffing' else row['start-stop'], 
        axis=1
    )

    # sniffing stops when next sniffing is none 
    bb['start-stop'] = bb.apply(
        lambda row: 'stop' if row['next_sniffing'] is None and row['sniffing'] == 'sniffing' else row['start-stop'], 
        axis=1
    )

    
    bb1 = bb.copy()
    bb1.drop(columns=['bounding_box', 'confidence', 'pred_classes', 'indexes_head', 'indexes_tank', 'bboxes_head', 'bboxes_tanks'], inplace=True)

    bb1 = bb1[bb1['start-stop'].isin(['start', 'stop'])]
    bb2 = bb1.copy()
    bb2.drop(columns=['prev_sniffing', 'next_sniffing'], inplace=True)
    bb3 = bb2.copy()
    bb3 = bb3[bb3['start-stop'] == 'start']

    bb4 = bb2.copy()
    bb4 = bb2[bb2['start-stop'] == 'stop']

    bb3.reset_index(drop=True, inplace=True)
    bb4.reset_index(drop=True, inplace=True)
    bb3['stop_timestamp'] = bb4['timestamp']

    bb3.drop(columns=['sniffing', 'start-stop'], inplace=True)
    bb3.head(25)


    bb4 = bb3.copy()
    bb4.rename(columns={'timestamp': 'start_timestamp'}, inplace=True)

    bb4.drop(columns=['indexes_nose', 'bboxes_nose'], inplace=True)


    # if value in stop_timestamp is NaN, then set it to the end of the video duration
    # convert duration of the video to stop_timestamp using frame count and fps

    frame_count = bb4['frame_number'].max() 
    last_timestamp = frame_count / fps  


    # Alternatively, you can use moviepy to get the duration of the video
    last_timestamp =   mp.VideoFileClip(input_video_path).duration 

    # Ensure that the last row's stop_timestamp is set to the last timestamp in the video

    if pd.isna(bb4.loc[bb4.index[-1], 'stop_timestamp']):
        bb4.loc[bb4.index[-1], 'stop_timestamp'] = last_timestamp

        
    # Convert start_timestamp and stop_timestamp to float
    bb4['start_timestamp'] = bb4['start_timestamp'].astype(float)
    bb4['stop_timestamp'] = bb4['stop_timestamp'].astype(float)
    # Calculate the duration of each sniffing event
    bb4['duration'] = bb4['stop_timestamp'] - bb4['start_timestamp']

    # calculate the last timestamp in the video
    last_timestamp = bb4['start_timestamp'].max() if not bb4['start_timestamp'].empty else 0

    if pd.isna(bb4.loc[bb4.index[-1], 'stop_timestamp']):
        bb4.loc[bb4.index[-1], 'stop_timestamp'] = last_timestamp

    bb4['stop_timestamp'] = bb4['stop_timestamp'].astype(float)
    bb4['start_timestamp'] = bb4['start_timestamp'].astype(float)
    bb4['duration'] = bb4['stop_timestamp'] - bb4['start_timestamp']
    bb4.head(10)
    return bb, bb2, bb4


# FUNCTION CSV 

In [106]:

def save_csv(foldername = f, vid_nr = vid_nr, bb = bb, bb2 = bb2, bb4 = bb4):

    csv_name = vid_nr + '_sniffing_start_stop_Complete.csv'
    output_folder_csv_name = os.path.join(f, csv_name)
    bb.to_csv(output_folder_csv_name , index=False)

    csv_name2 = vid_nr + '_sniffing_start_stop.csv'
    output_folder_csv_name = os.path.join(f, csv_name2)
    bb2.to_csv(output_folder_csv_name, index=False)

    csv_name3 = vid_nr + '_sniffing_just_startANDSTOPTIMES.csv'
    output_folder_csv_name = os.path.join(f, csv_name3)
    bb4.to_csv(output_folder_csv_name, index=False)

    print(vid_nr)


# FUNCTION CREATE SNIPPETS

In [107]:
def create_snippet(video_path, start, stop, output_path):
    video = mp.VideoFileClip(video_path)
    snippet = video.subclipped(start, stop).without_audio() 
    snippet.write_videofile(output_path, codec='libx264')
  

# EXECUTE EVERYTHING 

In [ ]:

for vid  in list_to_do:
    input_video_path = f"Videos_Cut/DSCN{vid}.mp4"
    vid_nr = input_video_path[-8:-4]
    print("Video number:", vid_nr)
    
    f = "SniffingSnippetsIsabelle/"
    output_video_name = vid_nr + 'processed_output.mp4'
    output_video_path = os.path.join(f, output_video_name)

    # duration of the video in seconds
    input_video_duration = mp.VideoFileClip(input_video_path).duration
    print("Input video duration:", input_video_duration, "seconds")

    bounding_box_location_tank_head, fps = process_video(input_video_path, output_video_path)
    bb, bb2 , bb4 = start_stop_sniffing(bounding_box_location_tank_head)

    save_csv(foldername = f, vid_nr = vid_nr, bb = bb, bb2 = bb2, bb4 = bb4)


    # Create snippets for each start-stop pair
    for index, row in bb4.iterrows():   
        
        start_time = row['start_timestamp']
        stop_time = row['stop_timestamp']              

        
        output_path = os.path.join(f, f"{vid_nr}_snippet_{index}.mp4")

        create_snippet(input_video_path, start_time, stop_time, output_path)



Video number: 2630
Input video duration: 26.09 seconds
[05/27 21:42:25 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...


INFO:fvcore.common.checkpoint:[Checkpointer] Loading from ./output/model_final.pth ...


2630
MoviePy - Building video SniffingSnippetsIsabelle/2630_snippet_0.mp4.
MoviePy - Writing video SniffingSnippetsIsabelle/2630_snippet_0.mp4



MoviePy - Done !
MoviePy - video ready SniffingSnippetsIsabelle/2630_snippet_0.mp4


MoviePy - Building video SniffingSnippetsIsabelle/2630_snippet_1.mp4.
MoviePy - Writing video SniffingSnippetsIsabelle/2630_snippet_1.mp4



...

MoviePy - Building video SniffingSnippetsIsabelle/2645_snippet_37.mp4.
MoviePy - Writing video SniffingSnippetsIsabelle/2645_snippet_37.mp4



MoviePy - Done !
MoviePy - video ready SniffingSnippetsIsabelle/2645_snippet_37.mp4
MoviePy - Building video SniffingSnippetsIsabelle/2645_snippet_38.mp4.
MoviePy - Writing video SniffingSnippetsIsabelle/2645_snippet_38.mp4



MoviePy - Done !
MoviePy - video ready SniffingSnippetsIsabelle/2645_snippet_38.mp4


MoviePy - Building video SniffingSnippetsIsabelle/2645_snippet_39.mp4.
MoviePy - Writing video SniffingSnippetsIsabelle/2645_snippet_39.mp4



MoviePy - Done !
MoviePy - video ready SniffingSnippetsIsabelle/2645_snippet_39.mp4


[05/27 21:35:48 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...


INFO:fvcore.common.checkpoint:[Checkpointer] Loading from ./output/model_final.pth ...


,frame_number,timestamp,bounding_box,confidence,pred_classes,indexes_head,indexes_tank,indexes_nose,bboxes_head,bboxes_tanks,bboxes_nose,sniffing,prev_sniffing,next_sniffing,start-stop
762,762,25.425400,"[[318.9358215332031, 506.448974609375, 426.247...","[0.9008474946022034, 0.900312066078186, 0.8969...","[2, 1, 2, 2, 2, 4, 2, 2]",[1],"[0, 2, 3, 4, 6, 7]",[5],"[[354.86761474609375, 99.2551498413086, 534.25...","[[318.9358215332031, 506.448974609375, 426.247...","[[449.24609375, 266.8948059082031, 497.4188842...",sniffing,sniffing,sniffing,continue
763,763,25.458767,"[[317.8729553222656, 506.0529479980469, 425.94...","[0.9043141603469849, 0.8849994540214539, 0.858...","[2, 2, 1, 2, 2, 1, 4, 2]","[2, 5]","[0, 1, 3, 4, 7]",[6],"[[355.37420654296875, 94.81307983398438, 531.2...","[[317.8729553222656, 506.0529479980469, 425.94...","[[440.8772888183594, 269.70849609375, 499.1054...",sniffing,sniffing,sniffing,continue
764,764,25.492133,"[[317.6860656738281, 505.0966796875, 421.96160...","[0.9043645858764648, 0.8977053165435791, 0.886...","[2, 2, 2, 2, 1, 4, 2, 2]",[4],"[0, 1, 2, 3, 6, 7]",[5],"[[347.8482666015625, 88.65413665771484, 531.74...","[[317.6860656738281, 505.0966796875, 421.96160...","[[436.7652282714844, 265.4993591308594, 493.59...",sniffing,sniffing,sniffing,continue
765,765,25.525500,"[[314.8800354003906, 506.94378662109375, 428.5...","[0.9025813341140747, 0.8896710872650146, 0.886...","[2, 2, 2, 2, 1, 4, 2, 2]",[4],"[0, 1, 2, 3, 6, 7]",[5],"[[345.1199035644531, 79.24298858642578, 522.31...","[[314.8800354003906, 506.94378662109375, 428.5...","[[434.2934875488281, 259.1200866699219, 492.26...",sniffing,sniffing,sniffing,continue
766,766,25.558867,"[[316.53125, 505.4551086425781, 423.5974426269...","[0.891058087348938, 0.8800485730171204, 0.8784...","[2, 2, 2, 2, 1, 4, 4, 2]",[4],"[0, 1, 2, 3, 7]","[5, 6]","[[343.30352783203125, 75.50825500488281, 525.0...","[[316.53125, 505.4551086425781, 423.5974426269...","[[431.9200134277344, 260.65496826171875, 495.2...",sniffing,sniffing,sniffing,continue
767,767,25.592233,"[[314.9231872558594, 505.67669677734375, 423.9...","[0.8991939425468445, 0.8810976147651672, 0.848...","[2, 2, 1, 2, 2, 4, 2, 4, 1]","[2, 8]","[0, 1, 3, 4, 6]","[5, 7]","[[354.3250427246094, 76.87438201904297, 521.58...","[[314.9231872558594, 505.67669677734375, 423.9...","[[429.52423095703125, 263.2339172363281, 492.9...",sniffing,sniffing,sniffing,continue
768,768,25.625600,"[[356.0720520019531, 91.31932067871094, 523.65...","[0.9182637929916382, 0.9098331332206726, 0.893...","[1, 2, 2, 2, 2, 1, 4, 2, 2, 2]","[0, 5]","[1, 2, 3, 4, 7, 8, 9]",[6],"[[356.0720520019531, 91.31932067871094, 523.65...","[[316.5494689941406, 505.56109619140625, 422.2...","[[427.43951416015625, 273.7341003417969, 490.6...",sniffing,sniffing,sniffing,continue
769,769,25.658967,"[[317.7554931640625, 506.02587890625, 422.7510...","[0.9029914140701294, 0.8996021151542664, 0.883...","[2, 2, 2, 1, 2, 4, 2, 2, 2, 1]","[3, 9]","[0, 1, 2, 4, 6, 7, 8]",[5],"[[353.0123596191406, 97.14250946044922, 538.53...","[[317.7554931640625, 506.02587890625, 422.7510...","[[429.57733154296875, 277.9071044921875, 475.5...",sniffing,sniffing,not_sniffing,continue
770,770,25.692333,"[[1035.053955078125, 264.3686218261719, 1115.4...","[0.9074544906616211, 0.9058253169059753, 0.897...","[2, 2, 2, 1, 2, 4, 2]",[3],"[0, 1, 2, 4, 6]",[5],"[[351.03314208984375, 103.86978912353516, 518....","[[1035.053955078125, 264.3686218261719, 1115.4...","[[425.7305908203125, 274.8208312988281, 493.82...",not_sniffing,sniffing,not_sniffing,stop
771,771,25.725700,"[[1166.5145263671875, 356.42218017578125, 1255...","[0.8981720209121704, 0.8980739712715149, 0.870...","[2, 2, 2, 1, 2, 4, 2]",[3],"[0, 1, 2, 4, 6]",[5],"[[347.2008972167969, 103.38174438476562, 513.9...","[[1166.5145263671875, 356.42218017578125, 1255...","[[422.1845703125, 268.92132568359375, 491.9032...",not_sniffing,not_sniffing,not_sniffing,continue


,frame_number,start_timestamp,stop_timestamp,duration
0,21,0.700700,0.800800,0.100100
1,31,1.034367,1.167833,0.133467
2,37,1.234567,1.267933,0.033367
3,39,1.301300,1.401400,0.100100
4,45,1.501500,1.701700,0.200200
5,55,1.835167,2.135467,0.300300
6,65,2.168833,2.202200,0.033367
7,67,2.235567,2.535867,0.300300
8,77,2.569233,2.635967,0.066733
9,96,3.203200,3.336667,0.133467


2630


MoviePy - Building video SniffingSnippetsIsabelle/2630_snippet_0.mp4.
MoviePy - Writing video SniffingSnippetsIsabelle/2630_snippet_0.mp4



MoviePy - Done !
MoviePy - video ready SniffingSnippetsIsabelle/2630_snippet_0.mp4


MoviePy - Building video SniffingSnippetsIsabelle/2630_snippet_1.mp4.
MoviePy - Writing video SniffingSnippetsIsabelle/2630_snippet_1.mp4



MoviePy - Done !
MoviePy - video ready SniffingSnippetsIsabelle/2630_snippet_1.mp4


MoviePy - Building video SniffingSnippetsIsabelle/2630_snippet_2.mp4.
MoviePy - Writing video SniffingSnippetsIsabelle/2630_snippet_2.mp4



...

MoviePy - Building video SniffingSnippetsIsabelle/2630_snippet_68.mp4.
MoviePy - Writing video SniffingSnippetsIsabelle/2630_snippet_68.mp4



MoviePy - Done !
MoviePy - video ready SniffingSnippetsIsabelle/2630_snippet_68.mp4


MoviePy - Building video SniffingSnippetsIsabelle/2630_snippet_69.mp4.
MoviePy - Writing video SniffingSnippetsIsabelle/2630_snippet_69.mp4



MoviePy - Done !
MoviePy - video ready SniffingSnippetsIsabelle/2630_snippet_69.mp4
MoviePy - Building video SniffingSnippetsIsabelle/2630_snippet_70.mp4.
MoviePy - Writing video SniffingSnippetsIsabelle/2630_snippet_70.mp4



MoviePy - Done !
MoviePy - video ready SniffingSnippetsIsabelle/2630_snippet_70.mp4
